## Connecting to Federated Query Executed System

### Presto
``
conn = presto.Connection(host="presto", port=8080, user="demo")
cur = conn.cursor()
``
### Drill
``drill = PyDrill(host='drill', port=8047)``
### Spark
TODO
### Hive
TODO


In [1]:
%pip install pathlib
%pip install jinja2
%pip install git+git://github.com/PythonicNinja/pydrill.git

  Using cached pathlib-1.0.1.tar.gz (49 kB)
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14346 sha256=78055564458b1c70a5a023ae01568d86ab4cc3497457dfe9595284495b0bcebe
  Stored in directory: /home/sathi/.cache/pip/wheels/59/02/2f/ff4a3e16a518feb111ae1405908094483ef56fec0dfa39e571
Successfully built pathlib
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Cloning git://github.com/PythonicNinja/pydrill.git to /tmp/pip-req-build-5b2nnmzt
  Running command git clone -q git://github.com/PythonicNinja/pydrill.git /tmp/pip-req-build-5b2nnmzt
  Created wheel for pydrill: filename=pydrill-0.3.4-py2.py3-none-any.whl size=18901 sha256=d0549dd95f32a73fc21575d2d9ae260f72076180902cc93a7dc3df6714471752
  Stored in directory: /tmp/pip-ephem-wheel-cache-9qz0d3ra/wheels/de/02/ce/322e1d74e96724d4437565c5a5740093a6cf66f51af96b9cd7
Successfully built pydrill
Note: you may need to restart the ker

In [4]:
!pip install presto-python-client

In [1]:
from pathlib import Path
import jinja2
import prestodb.dbapi as presto
from pydrill.client import PyDrill
import pandas as pd
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [2]:
# configure sources
sources={
    "li_source":"pg1",
    "o_source":"pg2",
    "c_source":"pg3",
    "pa_source":"pg4",
    "s_source":"pg1",
    "ps_source":"pg2",
    "n_source":"pg3",
    "r_source":"pg4",
}

def get_sources_config(scale,**kwargs):
    """Returns a dict with tabels maped to sources"""
    mapped = {
        "lineitem":f"{kwargs['li_source']}.public.pg_{scale}_lineitem",
        "orders":f"{kwargs['o_source']}.public.pg_{scale}_orders",
        "customer":f"{kwargs['c_source']}.public.pg_{scale}_customer",
        "part":f"{kwargs['pa_source']}.public.pg_{scale}_part",
        "supplier":f"{kwargs['s_source']}.public.pg_{scale}_supplier",
        "partsupp":f"{kwargs['ps_source']}.public.pg_{scale}_partsupp",
        "nation":f"{kwargs['n_source']}.public.pg_{scale}_nation",
        "region":f"{kwargs['r_source']}.public.pg_{scale}_region"
    }
    return mapped


config = get_sources_config(scale="sf1",**sources)

In [3]:
def get_queries():
    return [q for q in Path("./queries").iterdir()]

def render_queries(sources_config,queries_path):
    """Returns a dict where the key is the tpch query name and the value is the rendered query"""
    result = {}
    for path in queries_path:
        key = path.name.split(".")[0]
        value = jinja2.Template(path.read_text()).render(**sources_config)
        result[key]=value
    return result

query_dict = render_queries(sources_config=config,queries_path=get_queries())

In [4]:
class Presto():
    def __init__(self,host="presto",port=8080,user="demo"):
        self.conn = presto.Connection(host="presto", port=8080, user="demo")
        self.cur = self.conn.cursor()
    
    def run_query(self,sql,show=False):
        self.cur.execute(sql)
        result = self.cur.fetchall()
        if show:
            return result

In [5]:
class Drill(PyDrill):
    def __init__(self,**kwargs):
        PyDrill.__init__(self, **kwargs)
        
    def run_query(self,sql,timeout=100000,show=False):
        if show:
            return self.query(sql,timeout=timeout).to_dataframe()
        return self.query(sql,timeout=timeout)

In [6]:
conn = presto.Connection(host="presto", port=8080, user="demo")
cur = conn.cursor()

In [7]:
drill = Drill(host='drill', port=8047)

In [8]:
drill.is_active()

HEAD http://drill:8047/ [status:N/A request:0.021s]
Traceback (most recent call last):
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/sathi/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 394, in _make_requ

HEAD http://drill:8047/ [status:N/A request:0.007s]
Traceback (most recent call last):
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/sathi/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/sathi/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 394, in _make_requ

False

In [33]:
if not drill.is_active():
    raise ValueError('Please run Drill first')

presto = Presto()

HEAD http://drill:8047/ [status:N/A request:0.010s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  Fil

HEAD http://drill:8047/ [status:N/A request:0.003s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.request(method, url, **httplib_request_kw)
  Fil

ValueError: Please run Drill first

## How Does Presto Execute a Query?

If you are curious about what Presto translate a SQL query to and what it will run, you can you `EXPLAIN`:

In [28]:
conn = presto.Connection(host="presto", port=8080, user="demo")
cur = conn.cursor()

In [34]:
cur.execute('''
 SELECT
     l_orderkey,
     sum(l_extendedprice * (1 - l_discount)) as revenue,
     o_orderdate,
     o_shippriority
 FROM
     pg2.public.pg_sf1_customer,
     pg3.public.pg_sf1_orders,
     pg3.public.pg_sf1_lineitem
 WHERE
     c_mktsegment = 'BUILDING'
     AND c_custkey = o_custkey
     AND l_orderkey = o_orderkey
     AND o_orderdate < date '1995-03-15'
     AND l_shipdate > date '1995-03-15'
 GROUP BY
     l_orderkey,
     o_orderdate,
     o_shippriority
 ORDER BY
     revenue desc,
     o_orderdate
 LIMIT 10
 ''')

ConnectionError: HTTPConnectionPool(host='presto', port=8080): Max retries exceeded with url: /v1/statement (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1e38d45250>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
presto_res = cur.fetchall()

In [ ]:
# presto_df = pd.DataFrame(presto_res,columns=["l_orderkey","revenue","o_orderdate","o_shippriority"])
# presto_df["l_orderkey"]=presto_df["l_orderkey"].astype(int)
# presto_df["o_shippriority"]=presto_df["o_shippriority"].astype(int)
# presto_df["revenue"]=presto_df["revenue"].astype(float)
# presto_df.dtypes

Execute `docker exec drill /bin/sh -c "./create_plugins.sh"` to add storage to drill. This should be done everytime you restart drill container, as there is no persistant storage where the config is saved. 

This can be improved by finding where the configuration are written ang mounting a volume there.

In [ ]:
# drill_res = drill.query('''SELECT
#     l_orderkey,
#     sum(l_extendedprice * (1 - l_discount)) as revenue,
#     o_orderdate,
#     o_shippriority
# FROM
#     pg2.public.pg_sf1_customer,
#     pg3.public.pg_sf1_orders,
#     pg3.public.pg_sf1_lineitem
# WHERE
#     c_mktsegment = 'BUILDING'
#     AND c_custkey = o_custkey
#     AND l_orderkey = o_orderkey
#     AND o_orderdate < date '1995-03-15'
#     AND l_shipdate > date '1995-03-15'
# GROUP BY
#     l_orderkey,
#     o_orderdate,
#     o_shippriority
# ORDER BY
#     revenue desc,
#     o_orderdate
# LIMIT 10''', timeout=1000)

In [ ]:
# drill_df = drill_res.to_dataframe()
# drill_df["l_orderkey"]=presto_df["l_orderkey"].astype(int)
# drill_df["o_shippriority"]=presto_df["o_shippriority"].astype(int)
# drill_df["revenue"]=presto_df["revenue"].astype(float)
# drill_df.dtypes

In [ ]:
def are_equal(df1,df2):
    m = df1.merge(df2, how='outer', indicator=True)
    return len(m[m["_merge"]!="both"]) == 0

In [ ]:
# are_equal(drill_df,presto_df)

# Benchmark

In [ ]:
def benchmark(system,query,iterations=10):
    sum = 0
    count = 0
    for _ in range(iterations):
        count = count + 1
        start = time.time()
        system.run_query(query)
        end = time.time()
        sum = sum + (end - start)
    return sum/count

In [ ]:
# q2, q9, q11 -> cannot run as ps_supplycost column by partsupp is not shown by presto
presto_benchmark_results=[]

presto_benchmark_results.append(benchmark(presto,query_dict["q1"],iterations=10))
presto_benchmark_results.append(benchmark(presto,query_dict["q3"],iterations=10))
presto_benchmark_results.append(benchmark(presto,query_dict["q4"],iterations=10))
presto_benchmark_results.append(benchmark(presto,query_dict["q5"],iterations=10))
presto_benchmark_results.append(benchmark(presto,query_dict["q7"],iterations=10))
print(presto_benchmark_results)

In [ ]:
drill_benchmark_results=[]

drill_benchmark_results.append(benchmark(drill,query_dict["q1"],iterations=10))
drill_benchmark_results.append(benchmark(drill,query_dict["q3"],iterations=10))
drill_benchmark_results.append(benchmark(drill,query_dict["q4"],iterations=10))
drill_benchmark_results.append(benchmark(drill,query_dict["q5"],iterations=10))
drill_benchmark_results.append(benchmark(drill,query_dict["q7"],iterations=10))
print(drill_benchmark_results)

In [ ]:
drill_benchmark_results.append(benchmark(drill,query_dict["q3"],iterations=10))

In [ ]:
drill_benchmark_results.append(benchmark(drill,query_dict["q4"],iterations=10))

In [ ]:
drill = Drill(host='drill', port=8047)
# drill_benchmark_results.append(benchmark(drill,query_dict["q5"],iterations=10))
drill_benchmark_results.append(0)

In [ ]:
drill_benchmark_results.append(benchmark(drill,query_dict["q7"],iterations=10))

In [ ]:
drill_benchmark_results

In [ ]:
presto_benchmark_results

In [ ]:
%pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
labels = ["q1","q3","q4","q5","q7"]
x = np.arange(len(labels))
width = 0.35 
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, presto_benchmark_results, width, label='Presto')
rects2 = ax.bar(x + width/2, drill_benchmark_results, width, label='Drill')

ax.set_ylabel('Processing time (s)')
ax.set_title('TPC-H Queries')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()

plt.show()

## Create a Table in MySQL

Let's use the MySQL client to create the table. Then we will switch to Presto to manipulate the data:

In [ ]:
# import MySQLdb

# mysql = MySQLdb.connect(host="mysql", user="root", passwd="mysql")
# cur = mysql.cursor()
# cur.execute("CREATE DATABASE IF NOT EXISTS presto")
# cur.fetchall()
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS presto.events (event LONGTEXT)
#     CHARSET utf8mb4 ENGINE=InnoDB
# """)
# cur.fetchall()
# cur.execute("DESC presto.events")
# for row in cur.fetchall():
#     print("{table}: [{props}]".format(
#         table=row[0],
#         props=', '.join(str(i) for i in row[1:])))

## Load Data in MySQL

Let's now load data from [GH Archive](http://www.gharchive.org/) into MySQL and MongoDB.
Each file from GH Archive contains lines of JSON structs that represent events from the public GitHub timeline, for example repository creation or code push.

Now that the table is create in MySQL, we can insert rows with Presto by using the existing `conn` object created above. You can open http://localhost:8080 to see the execution Presto queries.

In [ ]:
# import gzip
# import io
# import json
# import re
# import requests

# # Load events happening between 4-5pm.
# # Feel free to load more hours or more days.
# # We limit the dataset to one hour here to not overload
# # the machine that will run the queries as this tutorial
# # is expected to run on a laptop.
# # It is going to take some time. For the demo, i pre-loaded
# # the data with the mysql client to avoid the overhead of creating
# # Python objects.
# zdata = requests.get("https://data.gharchive.org/2015-04-28-16.json.gz")
# data = gzip.decompress(zdata.content)
# rows = []

# # load ``ROW_COUNT`` rows. Feel free to set a greater value if it
# # works well in your environment. Using a small value on purpose
# # to avoid loading data for a long time.
# ROW_COUNT = 1000
# cur = conn.cursor()
# for n, line in enumerate(io.BytesIO(data)):
#     row = line.strip().decode('utf8')
#     sql = "INSERT INTO mysql.presto.events (event) VALUES ('{}')".format(row.replace("'", "''"))
#     cur.execute(sql)
#     cur.fetchall()
#     if n == ROW_COUNT - 1:
#         break

In [ ]:
# cur = conn.cursor()
# cur.execute("SELECT json_extract(json_parse(event), '$.type') FROM mysql.presto.events TABLESAMPLE BERNOULLI (1) LIMIT 1")
# cur.fetchall()

In [ ]:
# cur = conn.cursor()
# cur.execute("""
#     SELECT ev_type, repo_name, count(*) FROM (
#       SELECT
#         TRY(json_extract_scalar(ev, '$.repo.name')) as repo_name,
#         TRY(json_extract_scalar(ev, '$.type')) as ev_type FROM (
#           SELECT try(json_parse(event)) as ev FROM mysql.presto.events))
#     WHERE repo_name is not null and ev_type = 'PushEvent'
#     GROUP BY ev_type, repo_name
#     ORDER BY 3 DESC
#     LIMIT 10
# """)
# rows = cur.fetchall()
# import pandas as pd
# from IPython.display import display

# print(rows)
# df = pd.DataFrame(sorted(rows, key=lambda x: x[2], reverse=True))
# display(df)

In [ ]:
# cur = conn.cursor()
# cur.execute("CREATE TABLE mongodb.events.all AS SELECT * FROM mysql.presto.events")
# cur.fetchall()